# Column Transformer

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

**As you can see we have 10 null values in fever column**
- To Take Care of null values we have SimpleImputer.

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

In [6]:
X = df.drop('has_covid', axis=1)
y = df.has_covid

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=101)

In [8]:
X_train

,age,gender,fever,cough,city
61,81,Female,98.0,Strong,Mumbai
47,18,Female,104.0,Mild,Bangalore
54,60,Female,99.0,Mild,Mumbai
57,49,Female,99.0,Strong,Bangalore
2,42,Male,101.0,Mild,Delhi
...,...,...,...,...,...
63,10,Male,100.0,Mild,Bangalore
70,68,Female,101.0,Strong,Delhi
81,65,Male,99.0,Mild,Delhi
11,65,Female,98.0,Mild,Mumbai


# Firstly we see the problem We have without Column Transformer

In [11]:
# We have null values in fever, deal with them with SimpleImputer
si = SimpleImputer()

train_null_free_fever = si.fit_transform(X_train[['fever']])
test_null_free_fever = si.transform(X_test[['fever']])
train_null_free_fever.shape

(80, 1)

In [12]:
# Cough is Ordinal Column We will use OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])

In [13]:
X_train_transformed_cough = oe.fit_transform(X_train[['cough']])
X_test_transformed_cough = oe.transform(X_test[['cough']])
X_train_transformed_cough.shape

(80, 1)

In [14]:
# gender and city are nominal columns we will use OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse=False)

X_train_trans_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_trans_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_trans_gender_city.shape

(80, 4)

In [21]:
# Extract Age Column
X_train_age = X_train[['age']].values
X_test_age = X_test[['age']].age.values
X_train_age.shape

(80, 1)

In [24]:
# Now the final Transformed data is
X_train_transformed = np.concatenate((X_train_age,train_null_free_fever, X_train_transformed_cough,X_train_trans_gender_city),axis=1)
# X_test_transformed = np.concatenate((X_test_age,test_null_free_fever, X_test_transformed_cough,X_test_trans_gender_city),axis=1)

In [25]:
X_train_transformed.shape

(80, 7)

In [27]:
# X_train_transformed

# How to Solve Above Difficult Task with ColumnTransformer Within One Line

In [28]:
# Use Column Transformer
from sklearn.compose import ColumnTransformer

In [32]:
ct = ColumnTransformer([
    ("imputer", SimpleImputer(),['fever']),
    ("odinalencoder", OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
     ("onehotencoder",OneHotEncoder(drop='first',sparse=False),['gender','city'])],remainder='passthrough')

In [33]:
X_train_transformed = ct.fit_transform(X_train)

In [34]:
X_train_transformed.shape

(80, 7)